In [1]:
# IMPORTS GLOBAIS
import pandas as pd
import numpy as np
import os
import pickle
import mlflow.pyfunc

In [2]:
# MUDANDO DIRETÓRIO
os.chdir("c://Users//gufer//OneDrive//Documentos//FIAP//Fase_05//ML_Engineer_Datathon")
os.getcwd()

'c:\\Users\\gufer\\OneDrive\\Documentos\\FIAP\\Fase_05\\ML_Engineer_Datathon'

In [3]:
# IMPORTS LOCAIS
from src.recommendation_model.lgbm_ranker import LightGBMRanker

In [4]:
# LOADS
X_train = pd.read_parquet("data/train/X_train.parquet")
y_train = pd.read_parquet("data/train/y_train.parquet")
X_train_full = pd.read_parquet("data/train/X_train_full.parquet")
group_train = pd.read_parquet("data/train/group_train.parquet")

In [5]:
# INSTANCIANDO E TREINANDO O MODELO
model = LightGBMRanker()
model.train(X_train.values, y_train.values.ravel(), group_train["groupCount"].values)

In [6]:
# COLUNAS
CLIENT_FEATURES = [
    'userTypeFreq', 
    'isWeekend', 
    'dayPeriodFreq',
]
NEWS_FEATURES = [
    'relLocalState', 
    'relLocalRegion',
    'relThemeMain', 
    'relThemeSub',  
    'localStateFreq', 
    'localRegionFreq', 
    'themeMainFreq',
    'themeSubFreq',
]

client_features = X_train[CLIENT_FEATURES]
news_features = X_train[NEWS_FEATURES] 

client_features.columns, news_features.columns

(Index(['userTypeFreq', 'isWeekend', 'dayPeriodFreq'], dtype='object'),
 Index(['relLocalState', 'relLocalRegion', 'relThemeMain', 'relThemeSub',
        'localStateFreq', 'localRegionFreq', 'themeMainFreq', 'themeSubFreq'],
       dtype='object'))

In [7]:
# MONTA O DICIONÁRIO
input_data = {
    "client_features": client_features,
    "news_features": news_features
}

In [8]:
# MODELO LOCAL
predicoes = model.predict(input_data)
print("Predições:", predicoes)


Predições: [-1.88824921 -2.49841121 -2.4491163  ... -1.29674705 -1.9811297
 -1.82170664]


In [10]:
# FUNÇÃO PARA COMPATIBILIZAR PREDICT
def flatten_input(input_data: dict) -> pd.DataFrame:
    """
    Recebe um dicionário com as chaves 'client_features' e 'news_features',
    onde cada valor é um DataFrame, e retorna um único DataFrame com as colunas
    na ordem esperada pela assinatura do modelo.
    """
    client_df = input_data.get("client_features")
    news_df = input_data.get("news_features")
    
    if client_df is None or news_df is None:
        raise ValueError("O input deve conter 'client_features' e 'news_features'.")
    
    # Verifica se já são DataFrames; se não, converte
    if not isinstance(client_df, pd.DataFrame):
        client_df = pd.DataFrame(client_df)
    if not isinstance(news_df, pd.DataFrame):
        news_df = pd.DataFrame(news_df)
    
    # Concatena os DataFrames horizontalmente (colunas lado a lado)
    df = pd.concat([client_df, news_df], axis=1)
    
    # Define a ordem das colunas conforme a assinatura do modelo
    expected_columns = [
        'isWeekend', 
        'relLocalState', 'relLocalRegion', 'relThemeMain', 'relThemeSub', 
        'userTypeFreq', 'dayPeriodFreq', 
        'localStateFreq', 'localRegionFreq', 'themeMainFreq', 'themeSubFreq'
    ]
    
    # Reordena as colunas; se alguma coluna estiver faltando, será gerado um KeyError
    df = df[expected_columns]
    
    return df

In [12]:
# MODELO MLFLOW

# Setando o host
mlflow.set_tracking_uri("http://localhost:5001")

# Carrega o modelo a partir do MLflow
model = mlflow.pyfunc.load_model("models:/news-recommender-dev@champion")

# "Achata" o input para obter um único DataFrame
flattened_input = flatten_input(input_data)

# Realiza a predição utilizando o método predict do modelo carregado
predicoes = model.predict(flattened_input)
print("Predições:", predicoes)

#TODO: tomando erro porque modelo local e remoto devem estar diferentes. Promover modelo.

Predições: [27.31918627 27.20634579 28.06742317 ... 40.27345643 34.30191294
 35.22682051]
